In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic


groq_api_key=os.getenv("GROQ_API_KEY")

model= ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")
model_2 = ChatAnthropic(model="claude-3-sonnet-20240229")


### `Task is like generate tweet and linkedin post on same topics in parallel`

In [2]:
prompt1 = PromptTemplate(
    template="Generate a tweet about {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a linkedin post about this {topic}',
    input_variables=['topic']
)

In [3]:
parser = StrOutputParser()

### *Now build the parallel chain*

In [6]:
parallel_chain = RunnableParallel(
    {
        'tweet': RunnableSequence(prompt1, model, parser),
        'linkedin': RunnableSequence(prompt2, model, parser)
    }
)

In [7]:
parallel_chain.invoke(
    {
        'topic': "AI"
    }
)

{'tweet': 'Exploring the endless possibilities with AI, where innovation meets intelligence to shape a future where the impossible becomes possible. 🌟🤖 #ArtificialIntelligence #AITransformations #TechInnovation',
 'linkedin': "Exciting times ahead! I'm thrilled to share that Alibaba Cloud has recently launched Qwen, a state-of-the-art AI model designed to revolutionize the way we interact with technology. Qwen is not just another addition to the AI family; it's a leap forward in natural language processing, designed to assist, learn, and adapt to a wide array of tasks and interactions with unparalleled accuracy and efficiency.\n\nWith Qwen, we're setting new standards in AI interaction, aiming to make technology more accessible and intuitive for everyone. Whether you're looking to enhance your business processes, explore innovative solutions, or simply stay ahead in the tech game, Qwen is your partner for the future.\n\nStay tuned as we continue to unveil more features and applications

In [8]:
response = parallel_chain.invoke(
    {
        'topic': "machine Learning"
    }
)

In [9]:
response['tweet']

'"Unlocking the future with Machine Learning: where data meets destiny to create innovative solutions and drive change. #MachineLearning #TechInnovation"'

In [10]:
parallel_chain.get_graph().print_ascii()

        +-------------------------------+          
        | Parallel<tweet,linkedin>Input |          
        +-------------------------------+          
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP